<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_BAA_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 866.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 19.5 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = 'BAA_agg '

# 전략에 사용할 ETF 명 리스트에 담기
canary_etf = ['SPY', 'VEA', 'VWO', 'BND']
aggressive_etf = ['QQQ', 'VWO', 'VEA', 'BND']
defensive_etf = ['BIL', 'IEF', 'TLT', 'LQD', 'TIP', 'BND', 'DBC']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(canary_etf + aggressive_etf + defensive_etf))

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  11 of 11 completed


,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
2023-11-24,70.5600,104.2100,91.7600,389.5100,104.7200,89.8000,40.6000,92.2700,24.0200,455.3000,45.9100
2023-11-22,70.9000,104.6500,91.7500,390.0600,105.2600,90.8700,40.4700,92.7500,24.1800,455.0200,45.6100
2023-11-21,70.8400,104.5500,91.7100,388.4700,104.8400,90.5500,40.5800,92.7800,24.3900,453.2700,45.5200
2023-11-20,70.7400,104.7700,91.6800,390.7400,104.8100,90.5900,40.8200,92.6600,24.2500,454.2600,45.7000
2023-11-17,70.6400,104.4600,91.6800,386.0400,104.3900,90.0400,40.3900,92.5300,23.9500,450.7900,45.5000
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-22,69.9590,103.8680,87.5119,284.0248,102.3713,98.2278,37.5481,93.9751,25.1780,393.7022,41.1175
2022-11-21,69.6001,103.4505,87.4928,279.9922,101.4685,96.8533,37.4511,93.4401,24.9195,388.4745,40.5167
2022-11-18,69.6001,102.9068,87.4832,282.9024,101.6222,96.4468,37.8003,93.4985,25.0090,389.8922,40.8462
2022-11-17,69.7165,103.4310,87.4832,282.8925,101.6318,97.1146,38.0718,93.8292,25.1879,388.1299,40.7783


In [3]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} BAA_agg.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-11-26 BAA_agg ### ETF 현재-1-3-6-12 개월 가격

,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
today/2023-11-24,70.5600,104.2100,91.7600,389.5100,104.7200,89.8000,40.6000,92.2700,24.0200,455.3000,45.9100
1mo_ago/2023-10-25,68.0578,102.0692,91.3324,350.3400,98.7263,83.1644,37.8800,89.0119,24.8800,417.5500,42.0200
3mo_ago/2023-08-25,70.4913,104.4743,90.5691,363.4938,103.7958,94.3117,39.9045,92.9479,24.4100,438.4235,44.5558
6mo_ago/2023-05-25,71.0676,105.6003,89.4022,338.7643,103.8114,98.5211,38.6724,95.2852,22.4700,411.6634,44.3251
12mo_ago/2022-11-22,69.9590,103.8680,87.5119,284.0248,102.3713,98.2278,37.5481,93.9751,25.1780,393.7022,41.1175


In [4]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-11-26 BAA_agg ### ETF 1-3-6-12 개월 수익률

,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
1mo_ago/2023-10-25,3.6766,2.0974,0.4682,11.1806,6.0710,7.9788,7.1806,3.6603,-3.4566,9.0408,9.2575
3mo_ago/2023-08-25,0.0975,-0.2529,1.3149,7.1573,0.8904,-4.7838,1.7429,-0.7294,-1.5977,3.8494,3.0393
6mo_ago/2023-05-25,-0.7142,-1.3166,2.6373,14.9797,0.8753,-8.8520,4.9845,-3.1644,6.8981,10.6001,3.5757
12mo_ago/2022-11-22,0.8591,0.3293,4.8543,37.1394,2.2943,-8.5798,8.1281,-1.8144,-4.5992,15.6458,11.6556


In [5]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror.copy()
df_etf_ror_weight.iloc[0] *= 12
df_etf_ror_weight.iloc[1] *= 4
df_etf_ror_weight.iloc[2] *= 2

str_etf_ror_weight = str_today + str_strategy + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-11-26 BAA_agg ### ETF 1-3-6-12 개월 가중치 적용

,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
1mo_ago/2023-10-25,44.1189,25.1685,5.6188,134.1669,72.8525,95.7461,86.1668,43.9232,-41.4790,108.4900,111.0899
3mo_ago/2023-08-25,0.3899,-1.0118,5.2596,28.6290,3.5615,-19.1353,6.9716,-2.9175,-6.3908,15.3974,12.1570
6mo_ago/2023-05-25,-1.4285,-2.6332,5.2745,29.9593,1.7505,-17.7040,9.9689,-6.3287,13.7962,21.2002,7.1513
12mo_ago/2022-11-22,0.8591,0.3293,4.8543,37.1394,2.2943,-8.5798,8.1281,-1.8144,-4.5992,15.6458,11.6556


In [6]:
df_etf_momentum_score = (12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) + (ror_12mo)
df_etf_momentum_score.index = ['momentum_score']

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-11-26 BAA_agg ### ETF 별 1-3-6-12 개월 Momentum Score

,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
momentum_score,43.9394,21.8529,21.0073,229.8947,80.4588,50.3270,111.2353,32.8626,-38.6728,160.7333,142.0539


In [7]:
# 12개월 이동평균 생성 및 저장
etf_12mo_ma = df_etf.rolling(window=252).mean().loc[df_etf.index[251]]  # 252 개의 영업일은 대략 12개월
df_etf_12mo_ma = pd.DataFrame([etf_12mo_ma], columns=etf_order, index=['12mo_moving_average'])

# 12개월 이동평균 수익률 생성 및 저장
df_etf_12mo_ror = (df_etf.iloc[0]/df_etf_12mo_ma - 1)*100
df_etf_12mo_ror.index = ['12mo_ma_ror(%)']

df_etf_12mo_ma = pd.concat([df_etf_12mo_ma, df_etf_12mo_ror])

str_etf_12mo_ma = str_today + str_strategy + "### ETF 별 12 개월의 Moving Average 및 수익률"
display(Markdown(str_etf_12mo_ma))
display(df_etf_12mo_ma)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_12mo_ma}\n')
  df_etf_12mo_ma.to_csv(f, index=True)

### 2023-11-26 BAA_agg ### ETF 별 12 개월의 Moving Average 및 수익률

,BND,TIP,BIL,QQQ,LQD,TLT,VWO,IEF,DBC,SPY,VEA
12mo_moving_average,71.0533,105.2674,89.5103,333.0067,104.3410,97.8618,39.8540,94.6090,24.0046,417.5591,44.3046
12mo_ma_ror(%),-0.6943,-1.0045,2.5134,16.9676,0.3632,-8.2379,1.8717,-2.4723,0.0642,9.0385,3.6236


In [9]:
# 카나리아 자산군 0 미만 개수 세기
canary_etf_negative_count = (df_etf_momentum_score[canary_etf] < 0).sum().sum()
aggressive_portion = 0

if canary_etf_negative_count == 0:
  aggressive_portion = 100
else:
  aggressive_portion = 0


## 종목 출력하기
aggressive_etf_list_text = ''
defensive_etf_list_text = ''

# 카나리아 자산군 중 모멘텀스코어 0 미만 개수: 0
if canary_etf_negative_count == 0:
  momentum_scores_aggressive_top_1 = df_etf_momentum_score[aggressive_etf].transpose().nlargest(1, 'momentum_score').transpose()

  str_aggressive_momentum_score = str_today + str_strategy + "### 공격자산 Momentum Score"
  display(Markdown(str_aggressive_momentum_score))
  display(momentum_scores_aggressive_top_1)
  etf_columns = momentum_scores_aggressive_top_1.columns
  aggressive_etf_list_text = ' '.join([str(asset) for asset in etf_columns])

  df_etf_defensive_positive_index = []

# 0 미만 카나리아 자산군 개수: 1 이상
if canary_etf_negative_count >= 1:
  ma_12mo_defensive_top_3 = df_etf_12mo_ma.iloc[1:][defensive_etf].transpose().nlargest(3, '12mo_ma_ror(%)').transpose()

  str_defensive_12mo_ma = str_today + str_strategy + "### 안전자산 Momentum Score"
  display(Markdown(str_defensive_12mo_ma))
  display(ma_12mo_defensive_top_3)

  # 안전자산 중 0 이상인 것만 필터링하기
  filtered_index = ma_12mo_defensive_top_3.columns[(ma_12mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_12mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])

str_canary_text = "## 카나리아 자산군 중 모멘텀스코어 0 미만 개수: " + str(canary_etf_negative_count)
str_aggressive_text = "## 공격자산 비율: " + str(aggressive_portion) + " %"
display(Markdown(str_today[1:] + str_strategy))
display(Markdown(str_canary_text))
display(Markdown(str_aggressive_text))

str_aggressive_etf_list = '## 추천 공격자산(각 100%): '
if canary_etf_negative_count == 0:
  str_aggressive_etf_list += aggressive_etf_list_text

str_defensive_etf_list = '## 추천 안전자산(각 33.3%): '
if canary_etf_negative_count >= 1:
  str_defensive_etf_list += defensive_etf_list_text

str_defensive_etf_count = '## 안전자산 수익률 0 이상 개수'

display(Markdown(str_aggressive_etf_list))
display(Markdown(str_defensive_etf_list))

# 안전자산 개수 및 비율
str_etf_defensive_positive_index = '## 안전자산 투자 개수: ' + str(len(df_etf_defensive_positive_index))
display(Markdown(str_etf_defensive_positive_index))
str_etf_defensive_positive_portion = '## 안전자산 투자 비율: ' + str(round(len(df_etf_defensive_positive_index) * 1/3 * 100, 1)) + " %"
display(Markdown(str_etf_defensive_positive_portion))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_canary_text}\n')
  f.write(f'{str_aggressive_text}\n')
  f.write(f'{str_aggressive_etf_list}\n')
  f.write(f'{str_defensive_etf_list}\n')
  f.write(f'{str_etf_defensive_positive_index}\n')
  f.write(f'{str_etf_defensive_positive_portion}')

### 2023-11-26 BAA_agg ### 공격자산 Momentum Score

,QQQ
momentum_score,229.8947


## 2023-11-26 BAA_agg 

## 카나리아 자산군 중 모멘텀스코어 0 미만 개수: 0

## 공격자산 비율: 100 %

## 추천 공격자산(각 100%): QQQ

## 추천 안전자산(각 33.3%): 

## 안전자산 투자 개수: 0

## 안전자산 투자 비율: 0.0 %